# Panel Data

Sometimes, data comes in such a way that many observations share certain common features. For example, several measurements can be made in the same location, under the same condition, or for the same subject. To understand the data and extract meaningful insights, we often need to aggregate these observations. This is where the groupby() function comes into play.

## Exploring Panel Data

As always, let's start by importing pandas and loading our dataset. This time our conversion to datetime will be a bit different.

In [1]:
import pandas as pd

# Load the data
df = pd.read_csv("data/sec_data.csv")

# Look at dates
df.datadate

# Convert the 'datadate' column to a datetime object, specifying the format
df.datadate = pd.to_datetime(df.datadate, format="%d/%m/%Y")

df

,GVKEY,iid,datadate,tic,conm,cshtrd,prcod,prcld,prchd,prccd,exchg
0,1004,1,2023-01-03,AIR,AAR CORP,260279,45.09,44.21,45.5800,44.60,11
1,1004,1,2023-01-04,AIR,AAR CORP,258372,44.56,44.56,45.6600,45.24,11
2,1004,1,2023-01-05,AIR,AAR CORP,132574,44.86,44.50,45.0400,44.82,11
3,1004,1,2023-01-06,AIR,AAR CORP,301259,45.20,45.20,46.5200,46.09,11
4,1004,1,2023-01-09,AIR,AAR CORP,372930,46.84,45.94,47.1500,46.18,11
...,...,...,...,...,...,...,...,...,...,...,...
9954,3358,2,2023-03-27,CMTL,COMTECH TELECOMMUN,39911,12.28,12.21,12.4799,12.25,14
9955,3358,2,2023-03-28,CMTL,COMTECH TELECOMMUN,47057,12.15,12.01,12.2150,12.03,14
9956,3358,2,2023-03-29,CMTL,COMTECH TELECOMMUN,65026,12.26,11.94,12.2600,12.11,14
9957,3358,2,2023-03-30,CMTL,COMTECH TELECOMMUN,49142,12.36,12.03,12.3661,12.12,14


We'll stop short of setting the index as our datetime value though. This is because an index must have unique values, and because this panel data contains lots of different company stocks for just one quarter of a year, we'll see the same date lots of times.

In [2]:
# How many unique dates exist in the data frame
df.datadate.nunique()

# Last date in the dataset
df.datadate.max()

# First date in the dataset
df.datadate.min()

Timestamp('2023-01-03 00:00:00')

Let's explore this panel data a bit more, to answer some simple questions:

- How many companies are considered in the data
- How many stocks are considered in the data 
- Which exchanges are considered in the data
- Which exchanges appear most


In [3]:
print("Unique companies", df.conm.nunique())
print("Unique stocks:", df.tic.nunique())

# If we use unique() instead of nunique() we'll get the actual values
print("Unique exhanges:", df.exchg.unique())
print("Exchanges by appearance:", df.exchg.value_counts())

Unique companies 156
Unique stocks: 161
Unique exhanges: [11 14 12 19]
Exchanges by appearance: exchg
11    6301
14    3100
12     496
19      62
Name: count, dtype: int64


## Grouping


Grouping is a powerful way to manipulate panel data. Once you've grouped, you can call functions and they will be applied groupwise. The most common application of grouping is to calculate returns on a stock-by-stock basis, but there are many other uses!

In [4]:
# Returns calculation without grouping... Incorrect!
# Here we will erroneously consider a price from the stock above in the dataframe
df["returns"] = df.prccd.pct_change()
df.head(65)

# Returns calculation with grouping... Much better!
df["returns"] = df.groupby("tic").prccd.pct_change()
df.head(65)

# Identify the number of rows in each group
counts = df.groupby("tic").size()

# Subset a specific group
apple = df.groupby("tic").get_group("AAPL")


Let's see what else we can do with grouping. Recall that we had more stocks than companies. Let's see why that is by looking at how many unique stocks are issued by each company (using the `tic`). Then let's list those companies.

In [5]:

# Identify the unique stocks for each company
df.groupby("conm").tic.unique()

# To list the specific companies that have multiple stocks, we'll need a conditional

# First we create a series with the number of unique stocks for each company
unique_stocks = df.groupby("conm").tic.nunique()

# Then we index that series with a conditional
unique_stocks[unique_stocks > 1]


conm
BEL FUSE INC                2
BERKSHIRE HATHAWAY          2
BIO-RAD LABORATORIES INC    2
BROWN FORMAN CORP           2
U-HAUL HOLDING CO           2
Name: tic, dtype: int64

### Exercise: Excellent Exchanges

**Part 1** Identify the number of unique stocks traded on each exchange.

In [6]:
df.groupby("exchg").tic.nunique()

exchg
11    102
12      8
14     50
19      1
Name: tic, dtype: int64

**Part 2** Then identify any companies that trade on more than one exchange.

In [7]:
# First create a series with the number of unique exchanges for each company
unique_exchg = df.groupby("conm").exchg.nunique()

# Then we index that series with a conditional
unique_exchg[unique_exchg > 1]

conm
BIO-RAD LABORATORIES INC    2
Name: exchg, dtype: int64

## Aggregation

Aggregation functions like `mean()`, `median()`, `sum()`, `min()`, `max()`, `first()`, `last()` and `std()` can be applied to grouped data to give insights across panel data. Say we wanted the average daily return of each traded stock, or the max volume traded on any given day for each stock?

In [8]:
df.groupby("tic").returns.mean()

df.groupby("tic").prccd.min()

df.groupby("tic").prccd.first() # notice this is different to min()!

df.groupby("tic").conm.first() # first() is commonly used for aggregating like, non-numeric data


tic
AAL       AMERICAN AIRLINES GROUP INC
AAPL                        APPLE INC
ABM                ABM INDUSTRIES INC
ABT               ABBOTT LABORATORIES
ACU                  ACME UNITED CORP
                     ...             
UHAL.B              U-HAUL HOLDING CO
UIS                       UNISYS CORP
VAL                       VALARIS LTD
VBF                 INVESCO BOND FUND
VZ         VERIZON COMMUNICATIONS INC
Name: conm, Length: 161, dtype: object

Once we've done these sorts of aggregation, we're often curious to see who sits at the top or the bottom of the distribution. We can use `nlargest()` and its antonym here.

In [9]:
df.groupby("tic").returns.mean().nsmallest()

df.groupby("tic").cshtrd.max().nlargest()


tic
LUMN     315918300
AAPL     154211800
AMD      152049200
AAL       96213400
CMCSA     51720130
Name: cshtrd, dtype: int64

We can also group by multiple columns! This can be helpful when doing aggregation, for example, to find high performers in each month.

In [10]:
# First create a column to specify the month
df["month"] = df.datadate.dt.month_name()

# Then use it to group and aggregate for max closing price each month
df.groupby(["tic", "month"]).prccd.max()

tic   month   
AAL   February     17.06
      January      17.08
      March        16.59
AAPL  February    155.33
      January     145.93
                   ...  
VBF   January      16.77
      March        16.22
VZ    February     41.83
      January      42.19
      March        38.89
Name: prccd, Length: 482, dtype: float64

### Exercise: Good Days

Which two days of the week see the highest average close in this data set, and what is the average close for those days?  

In [11]:
df["day"] = df.datadate.dt.day_name()
df.groupby("day").prccd.mean().nlargest(2)

day
Tuesday    3009.616720
Monday     3001.989468
Name: prccd, dtype: float64

### Exercise: Trading Exchanges

Next identify the total trading volume of each exchange.

In [12]:
df.groupby("exchg").cshtrd.sum()

exchg
11    17462494850
12       10819766
14    14134028207
19       21345489
Name: cshtrd, dtype: int64

### Exercise: The 500 Club

For stocks that reached a closing price above 500, how many times in each month, did they acheive this?

In [13]:
df[df.prccd > 500].groupby(["tic", "month"]).size()

tic    month   
ATRI   February    19
       January     20
       March       23
BIO    March        2
BRK.A  February    19
       January     20
       March       23
CHE    February    15
       January      8
       March       23
COKE   February    19
       January      3
       March       22
dtype: int64